<a href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Cancer Detection

## Objective
* To build a dense prediction model
* To begin reading current papers in DNN research

## Deliverable
For this lab, you will turn in a notebook that describes your efforts at creating
a pytorch radiologist. Your final deliverable is a notebook that has (1) a deep network,
(2) method of calculating accuracy, 
(3) images that show the dense prediction produced by your network on the pos_test_000072.png image (index 172 in the validation dataset).
This is an image in the test set that your network will not have seen before.
This image, and the ground truth labeling, is shown below.
(And is contained in the downloadable dataset below).

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=200&tok=a8ac31&media=cs501r_f2016:pos_test_000072_output.png)
<img src="http://liftothers.org/dokuwiki/lib/exe/fetch.php?media=cs501r_f2016:pos_test_000072.png" width="200">


## Grading standards
Your notebook will be graded on the following:
* 40% Proper design, creation and debugging of a dense prediction network
* 20% Proper implementation of train/test set accuracy measure
* 20% Tidy visualizations of loss of your dense predictor during training
* 20% Test image output


## Data set
The data is given as a set of 1024×1024 PNG images. Each input image (in 
the ```inputs``` directory) is an RGB image of a section of tissue,
and there a file with the same name (in the ```outputs``` directory) 
that has a dense labeling of whether or not a section of tissue is cancerous
(white pixels mean “cancerous”, while black pixels mean “not cancerous”).

The data has been pre-split for you into test and training splits.
Filenames also reflect whether or not the image has any cancer at all 
(files starting with ```pos_``` have some cancerous pixels, while files 
starting with ```neg_``` have no cancer anywhere).
All of the data is hand-labeled, so the dataset is not very large.
That means that overfitting is a real possibility.

## Description
For a video including some tips and tricks that can help with this lab: [https://youtu.be/Ms19kgK_D8w](https://youtu.be/Ms19kgK_D8w)
For this lab, you will implement a virtual radiologist.
You are given images of possibly cancerous tissue samples, 
and you must build a detector that identifies where in the tissue cancer may reside.

---

### Part 0
Watch and follow video tutorial:

https://youtu.be/Ms19kgK_D8w

**TODO:**

* Watch tutorial

**DONE:**

### Part 1
Implement a dense predictor

In previous labs and lectures, we have talked about DNNs that classify an 
entire image as a single class. Here, however, we are interested in a more nuanced classification: 
given an input image, we would like to identify each pixel that is possibly cancerous. 
That means that instead of a single output, your network should output an “image”, 
where each output pixel of your network represents the probability that a pixel is cancerous.

___

Implement your network topology

Use the “Deep Convolution U-Net” from this paper: [(U-Net: Convolutional Networks for Biomedical Image Segmentation)](https://arxiv.org/pdf/1505.04597.pdf) 

![(Figure 1)](https://lh3.googleusercontent.com/qnHiB3B2KRxC3NjiSDtY08_DgDGTDsHcO6PP53oNRuct-p2QXCR-gyLkDveO850F2tTAhIOPC5Ha06NP9xq1JPsVAHlQ5UXA5V-9zkUrJHGhP_MNHFoRGnjBz1vn1p8P2rMWhlAb6HQ=w2400)

You should use existing pytorch functions (not your own Conv2D module), such as ```nn.Conv2d``` and ```nn.CrossEntropyLoss```;
you will also need the pytorch function ```torch.cat``` and ```nn.ConvTranspose2d```

```torch.cat``` allows you to concatenate tensors.
```nn.ConvTranspose2d``` is the opposite of ```nn.Conv2d```.
It is used to bring an image from low res to higher res.
[This blog](https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0) should help you understand this function in detail.

You can simplify the implementation of this lab by padding the feature maps as they pass through each convolution. This will make the concatenation process easier, though this is technically a departure from the cropping technique outlined in the orginal U-Net paper. 

Note that the simplest network you could implement (with all the desired properties)
is just a single convolution layer with two filters and no relu! 
Why is that? (of course it wouldn't work very well!)

**TODO:**

* Understand the U-Net architecture
* Understand concatenation of inputs from multiple prior layers
* Understand ConvTranspose
* Answer Question / Reflect on simplest network with the desired properties

**DONE:**


___
The intention of this lab is to learn how to make deep neural nets. We'll help you with the implementation of Dataset.
This code will download the dataset for you so that you are ready to use it and focus on network
implementation, losses and accuracies.

In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

WARNING: You may run into an error that says "RuntimeError: CUDA out of memory."

In this case, the memory required for your batch is larger than what the GPU is capable of. You can solve this problem by adjusting the image size or the batch size and then restarting the runtime. 

In [2]:
class CancerDataset(Dataset):
  def __init__(self, root, download=True, size=512, train=True):
    if download and not os.path.exists(os.path.join(root, 'cancer_data')):
      datasets.utils.download_url('http://liftothers.org/cancer_data.tar.gz', root, 'cancer_data.tar.gz', None)
      self.extract_gzip(os.path.join(root, 'cancer_data.tar.gz'))
      self.extract_tar(os.path.join(root, 'cancer_data.tar'))
    
    postfix = 'train' if train else 'test'
    root = os.path.join(root, 'cancer_data', 'cancer_data')
    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'inputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))
    self.label_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'outputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))

  @staticmethod
  def extract_gzip(gzip_path, remove_finished=False):
    print('Extracting {}'.format(gzip_path))
    with open(gzip_path.replace('.gz', ''), 'wb') as out_f, gzip.GzipFile(gzip_path) as zip_f:
      out_f.write(zip_f.read())
    if remove_finished:
      os.unlink(gzip_path)
  
  @staticmethod
  def extract_tar(tar_path):
    print('Untarring {}'.format(tar_path))
    z = tarfile.TarFile(tar_path)
    z.extractall(tar_path.replace('.tar', ''))

  def __getitem__(self,index):
    img = self.dataset_folder[index]
    label = self.label_folder[index]
    return img[0],label[0][0]
  
  def __len__(self):
    return len(self.dataset_folder)

In [3]:
# Since you will be using the output of one network in two places(convolution and maxpooling),
# you can't use nn.Sequential.
# Instead you will write up the network like normal variable assignment as the example shown below:
# You are welcome (and encouraged) to use the built-in batch normalization and dropout layer.


class DownBlock(nn.Module):
  def __init__(self, in_channels,out_channels):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels,out_channels, kernel_size=3, stride=1,padding=1)
    self.conv2 = nn.Conv2d(out_channels,out_channels, kernel_size=3, stride=1, padding=1)
    self.activation = nn.ReLU()
    self.pool = nn.MaxPool2d(2,2)

  def forward(self,input):
    o1 = self.activation(self.conv1(input))
    o2 = self.activation(self.conv2(o1))
    self.send_for_skip = o2
    o3 = self.pool(o2)
    return o3

class UpBlock(nn.Module):
  def __init__(self, in_channels, intermediate_channels, out_channels):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels,intermediate_channels, kernel_size=3, stride=1,padding=1)
    self.conv2 = nn.Conv2d(intermediate_channels,intermediate_channels, kernel_size=3, stride=1, padding=1)
    self.activation = nn.ReLU()
    self.convTr = nn.ConvTranspose2d(intermediate_channels, out_channels, 2, stride=2)

  def forward(self, input):
    o1 = self.activation(self.conv1(input))
    o2 = self.activation(self.conv2(o1))
    o3 = self.convTr(o2)
    return o3 

class CancerDetection(nn.Module):
  def __init__(self):
    super(CancerDetection, self).__init__()
    self.db1 = DownBlock(3,64)
    self.db2 = DownBlock(64,128)
    self.db3 = DownBlock(128,256)
    self.db4 = DownBlock(256,512)

    self.ub1 = UpBlock(512,1024,512)

    self.ub2 = UpBlock(1024,512,256)

    self.ub3 = UpBlock(512,256,128)

    self.ub4 = UpBlock(256,128,64)

    self.endconv1 = nn.Conv2d(128,64, kernel_size=1, stride=1, padding=0)

    self.endconv2 = nn.Conv2d(64,64, kernel_size=1, stride=1, padding=0)

    self.endconv3 = nn.Conv2d(64,2, kernel_size=1, stride=1, padding=0)
 
  def forward(self, input):
    o1 = self.db1(input)
    o2 = self.db2(o1)
    o3 = self.db3(o2)
    o4 = self.db4(o3)
    o5 = self.ub1(o4)
    o6 = self.ub2(torch.cat((o5,self.db4.send_for_skip),1))
    o7 = self.ub3(torch.cat((o6,self.db3.send_for_skip),1))
    o8 = self.ub4(torch.cat((o7,self.db2.send_for_skip),1))
    o9 = self.endconv1(torch.cat((o8,self.db1.send_for_skip),1))
    o10 = self.endconv2(o9)
    o11 = self.endconv3(o10)
    return o11



In [4]:
# Create your datasets, dataloaders and neural network as you have before
# Tip: Pass shuffle=True to constructor for training dataloader
train_dataset = CancerDataset('/temp/Cancer/', download=True)
val_dataset = CancerDataset('/temp/Cancer/', download=True, train=False)

bs = 4

print(len(train_dataset))

train_loader = DataLoader(train_dataset, batch_size = bs, pin_memory = True, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size = bs)

1342


In [5]:

print("start", torch.cuda.memory_allocated()/1e9)
testmodel = CancerDetection().cuda()
gc.collect()
#print(testmodel(train_dataset[0][0]).shape ,train_dataset[0][1].shape)
print("model loaeded", torch.cuda.memory_allocated()/1e9)
for x,y in train_loader:
  x,y=x.cuda(),y.cuda()
  print("x,y loaded", torch.cuda.memory_allocated()/1e9)
  print(x.shape,y.shape)
  t= testmodel(x)
  print("ran test input", torch.cuda.memory_allocated()/1e9)
  print(t.dtype, y.dtype)
  print(t.shape, y.shape)
  nn.CrossEntropyLoss()(t,y.long())
  break

del testmodel
del x
del y
del t
gc.collect()
print("end", torch.cuda.memory_allocated()/1e9)


start 0.0
model loaeded 0.125702144
x,y loaded 0.14247936
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
ran test input 3.58180864
torch.float32 torch.float32
torch.Size([4, 2, 512, 512]) torch.Size([4, 512, 512])
end 0.0


In [6]:
objective = nn.CrossEntropyLoss()
def do_it():
  gc.collect()
  model = CancerDetection()
  model = model.cuda()
  optimizer= optim.SGD(model.parameters(), lr = 1e-4)
  losses = []
  validations = []
  #print(torch.cuda.memory_allocated() / 1e9)
  num_epochs = 10
  loop = tqdm(total=len(train_loader)*num_epochs, position = 0)

  for epoch in range(num_epochs):
    for  batch, (x,y_truth) in enumerate(train_loader):
      x,y_truth = x.cuda(non_blocking=True), y_truth.cuda(non_blocking=True).long()

      optimizer.zero_grad()
      y_hat =  model(x)
      loss = objective(y_hat, y_truth)

      #assert loss - myobjective(y_hat, y_truth) < 1e-6, f"myloss {myobjective(y_hat, y_truth)} != loss {loss}"

      loss.backward()

      losses.append(loss.item())
      accuracy = 0

      #loop.set_description("batch:{} loss:{:.4f} val_loss:?".format(batch, loss.item()))
      loop.update(1)

      optimizer.step()

      if batch %100 == 0:
        val = np.mean( [objective(model(x.cuda(non_blocking=True)), y.cuda(non_blocking=True).long()).item() for x,y in validation_loader])
        validations.append((len(losses),val))


      loop.set_description("batch:{} loss:{:.4f} val_loss:{:.4f}".format(batch, loss.item(), validations[-1][1]))


  loop.close()
  plt.plot(losses,label="train")
  plt.plot(*zip(*validations),label="val")
  return losses, validations

In [7]:
do_it()

batch:259 loss:0.6859 val_loss:0.6909:   8%|▊         | 260/3360 [05:56<57:29,  1.11s/it]  

In [ ]:
# This is what was talked about in the video for memory management

def scope():
  try:
    #your code for calling dataset and dataloader
    gc.collect()
    print(torch.cuda.memory_allocated() / 1e9)
    
    #for epochs:
    # Call your model, figure out loss and accuracy
    
  except:
    __ITB__()
    
scope()



___

### Part 2

Plot performance over time

Please generate two plots:

 One that shows loss on the training and validation set as a function of training time. 

 One that shows accuracy on the training and validation set as a function of training time. 

 Make sure your axes are labeled!

**TODO:**

* Plot training/validation loss as function of training time (not epochs)
* Plot training/validation accuracy as function of training time (not epochs)

**DONE:**



In [ ]:
# Your plotting code here


**NOTE:**

Guessing that the pixel is not cancerous every single time will give you an accuracy of ~ 85%.
Your trained network should be able to do better than that (but you will not be graded on accuracy).
This is the result I got after 1 hour or training.

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=d23e0b&media=cs501r_f2016:training_accuracy.png)
![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=bb8e3c&media=cs501r_f2016:training_loss.png)

___

### Part 3

Generate at least 5 predictions on the pos_test_000072.png image and display them as images. These predictions should be made at a reasonable interval (e.g. every epoch). 

To do this, calculate the output of your trained network on the pos_test_000072.png image,
then make a hard decision (cancerous/not-cancerous) for each pixel.
The resulting image should be black-and-white, where white pixels represent things
you think are probably cancerous.

**TODO:**

**DONE:**



In [ ]:
# Code for testing prediction on an image
